# 3.1. Boosting - GMB
 -> 여러 개의 약한 학습기를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식
 
#### 과적합에서도 뛰어난 예측 성능을 가졌지만,  시간이 너무너무너무너무 오래 걸린다는 단점이 있다.

 -  AdaBoost(Adaptive boosting) : 오류 데이터에 가중치를 부여하면서 부스팅을 수행하는 대표적인 알고리즘.
 - GMB(Gradient Boost Machine)은 경사하강법 사용 : 반복 수행을 통해 오류를 최소화 할 수 있도록 가중치의 업데이트 값을 도출하는 기법
 
 
       -> GBM은 약한 학습기의 순차적인 예측 오류 보정을 통해 학습을 수행하므로 멀티 CPU 코어에서도 병렬처리가 지원되지 않는다.

### 사이킷런의 GradientBoostingClassifier 클래스 사용

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# features.txt 파일에는 피처 이름 index와 피처명이 공백으로 분리되어 있음. 이를 DataFrame으로 로드.
feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])

# 피처명 index를 제거하고, 피처명만 리스트 객체로 생성한 뒤 샘플로 10개만 추출
feature_name = feature_name_df.iloc[:, 1].values.tolist()
print('전체 피처명에서 10개만 추출:', feature_name[:10])
feature_name_df.head(10)

전체 피처명에서 10개만 추출: ['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X']


,column_index,column_name
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y
5,6,tBodyAcc-std()-Z
6,7,tBodyAcc-mad()-X
7,8,tBodyAcc-mad()-Y
8,9,tBodyAcc-mad()-Z
9,10,tBodyAcc-max()-X


In [2]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                           if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

pd.options.display.max_rows = 999
new_feature_name_df = get_new_feature_name_df(feature_name_df)
new_feature_name_df[new_feature_name_df['dup_cnt'] > 0]

def get_human_dataset():
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 feature명을 새롭게 수정하는 get_new_feature_name_df()를 이용하여 새로운 feature명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # [X, 피처셋] 칼럼명 feature_name에 train 피처셋과 test 피처셋 적용
    X_train = pd.read_csv('./human_activity/train/X_train.txt', sep='\s+', names=feature_name)
    X_test = pd.read_csv('./human_activity/test/X_test.txt', sep='\s+', names=feature_name)
    
    # [y, 레이블셋] 칼럼명 action에 train 레이블셋과 test 레이블셋 적용
    y_train = pd.read_csv('./human_activity/train/y_train.txt', sep='\s+', header=None, names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt', sep='\s+', header=None, names=['action'])
    
    
    # 로드된 train/test용 DataFrame을 모두 반환
    return X_train, X_test, y_train, y_test


In [3]:
from sklearn.ensemble import GradientBoostingClassifier
import time  # 수행시간 표현을 위해 import
import warnings
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

# GBM 수행 시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print("GBM 수행시간: {0:.1f} 초" .format(time.time() - start_time))

# GBM 정확도: 0.9389
# GBM 수행시간: 173.6 초

GBM 정확도: 0.9389
GBM 수행시간: 173.6 초


# GradientBoostingClassifier 하이퍼 파라미터 및 튜닝
 - n_estimators : weak learner의 개수. 순차적으로 오류를 보정하므로 개수가 많을수록 예측 성능이 일정 수준까지는 좋아진다. 기본값은 100
 - max_features : 기본적으로는 sqrt이다. 따라서 랜덤 포레스트의 트리를 분할하는 피처를 참조할 때 전체 피처가 아니라 sqrt(전체 피처 개수) 만큼 참조한다. 전체 피처가 16개라면 분할을 위해 4개 참조.
 - max_depth, max_samples_leaf와 같이 과적합을 개선하기 위해 사용되는 파라미터가 랜덤 포레스트에도 똑같이 적용.
 - GridSearchCV도 동일하게 적용된다는 것이다. 기본이 결정 트리이니까......
    
 - loss : 경사 하강법에서 사용할 비용 함수를 지정. 거의 대부분 기본값인 'deviance'를 그대로 적용.
 - learning_rate : 경사하강법에서 반드시 쓰이는 학습률. 0~1사이 값 지정. 기본값은 0.1 너무 작은 값 적용시 업데이트 되는 값이 작아져서 최소 오류 값을 찾아 예측 성능이 높아질 가능성 높음.
       -> learning_rate는 n_estimators와 상호 보완적으로 조합해 사용한다. learning_rate를 작게, n_estimators를 크게 하면 더 이상 성능이 좋아지지 않는 한계점까지 예측 성능이 조금씩 좋아질 수 있지만 시간이 너무 오래 걸린다.
 - subsample : weak learner가 학습에 사용하는 데이터의 샘플링 비율. 기본값은 1. 0.5면 학습 데이터의 50%, 학습. 과적합 염려시 1보다 작은 값으로 설정.   

In [4]:
# 교차검증(cv)를 2개로 제약 후 테스트

from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100, 500], 
    'learning_rate' : [0.05, 0.1]
}

grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

# 돌리는 데 1시간 걸림
# 돌리는 데 1시간 걸림
# 돌리는 데 1시간 걸림
# 돌리는 데 1시간 걸림
# 돌리는 데 1시간 걸림

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 18.7min finished


최적 하이퍼 파라미터:
 {'learning_rate': 0.05, 'n_estimators': 500}
최고 예측 정확도: 0.9014


In [5]:
# GridSearchCV를 이용해 최적으로 학습된 estimator로 테스트 데이터 예측 수행.

gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도: {0:.4f}'.format(gb_accuracy))

GBM 정확도: 0.9396
